# Agent V4: The Coordinator

A monolithic agent with sub-agent delegation and task orchestration. Instead of giving
the agent 40+ specialized tools, it declares sub-agents via `AgentSpec(sub_agents=[...])`.

The `OrchestratorAgent` auto-creates a `delegate` tool from the sub-agent specs.
The coordinator's prompt uses synchronous delegation only; async tools (`submit_task`,
`wait`) are unlocked in V5 by the prompt.

Extends the Skilled agent:
- file tools
- sandbox
- todo
- skill
- delegate (from sub-agents)
- conversion

**Prerequisite:** Run `scripts/db_ingest_bookstore_sqlite.sh` to create the bookstore database before running this notebook.

In [ ]:
from agentic_patterns.core.agents import AgentSpec, OrchestratorAgent
from agentic_patterns.core.workspace import clean_up_session

## Agent Definition

The coordinator extends the Skilled agent with sub-agent delegation via `AgentSpec(sub_agents=[...])`.
The `OrchestratorAgent` auto-creates `delegate`, `submit_task`, and `wait` tools from the specs.
Skills are still auto-discovered by the `OrchestratorAgent`.

In [ ]:
spec = AgentSpec.from_config("coordinator")
print(spec)

In [ ]:
clean_up_session()
agent = OrchestratorAgent(spec, verbose=True)

## System Prompt

The `OrchestratorAgent` builds the final system prompt by combining the base prompt,
sub-agent descriptions (for the `delegate` tool), and skill listings. The prompt is
assembled during `__aenter__` and available via the `system_prompt` property.

In [ ]:
async with agent:
    print(agent.system_prompt)

## Turn 1: Query the Database

Ask about data in the bookstore database. The coordinator should delegate to `sql_analyst`
rather than trying to query the database directly (it has no SQL tools of its own).

In [ ]:
async with agent:
    prompt_1 = """Query the bookstore database: how many books are there,
what genres are represented, and what is the average price per genre?
Save the per-genre results to a CSV file."""

    result_1 = await agent.run(prompt_1)

    print("\n--- Agent Output ---")
    print(result_1.output)

## Turn 2: Visualize, Report, and Convert

A natural request that mixes delegation and direct work. The coordinator should
delegate the chart to the `data_analyst` (which has visualization tools) and handle
the report writing and PDF conversion itself.

In [ ]:
async with agent:
    prompt_2 = """Write a markdown report summarizing the bookstore data from the previous query.
Include a bar chart showing the average price per genre (save it as /workspace/avg_price_by_genre.png
and embed it in the report). Save the report to /workspace/bookstore_report.md, then convert it to PDF."""

    result_2 = await agent.run(prompt_2)

    print("\n--- Agent Output ---")
    print(result_2.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    try:
        content = read_from_workspace(path)
    except UnicodeDecodeError:
        content = "<binary file>"
    print(f"--- {path} ---")
    print(content)
    print()